In [ ]:
import pandas as pd
from pathlib import Path

## 1. Cargar los datos 

In [4]:
# Directorio donde está el notebook
BASE_DIR = Path.cwd()
# Carpeta padre (un nivel arriba)
PARENT_DIR = BASE_DIR.parent

# Directorios de datos
directorio_14 = PARENT_DIR / "14. Mallado Total"
directorio_4 = PARENT_DIR / "04. BAlance Eléctrico"
directorio_2 = PARENT_DIR / "02. Precios"

# Cargar datos
df_potencia_d = pd.read_csv(f"{directorio_14}/MALLADO_TOTAL.csv", sep=',', encoding='latin-1')
df_balance_d= pd.read_csv(f"{directorio_4}/balance_REDUCIDO.csv", sep=';', decimal=',',encoding='latin-1')
df_precios_h = pd.read_csv(f"{directorio_2}/Precios_TOTAL.csv", sep=';', encoding='latin-1')

## 2. Transformar variables horaria PRECIOS en diaria (PRECIOS)

In [5]:
#_________________________________________________________________________________________________
 #_________________________FUNCIÓN transforma datos HORARIOS en DIARIOS_________________________   
 #________________________________________________________________________________________________


def generate_df_variable_diaria(df,variable):
    """
    Esta función toma un DataFrame con datos horarios y genera un DataFrame con datos diarios.
    Agrupa los datos por fecha y guarda las columnas horarias como variables, calcula la suma, la media, el máximo y el mínimo para cada día.

    Parámetros:
    df_variable_horaria (pd.DataFrame): DataFrame con datos horarios. Debe tener una columna 'Year', 'Month', 'Day', 'Time' y 'Variable'.
    Retorna:
    pd.DataFrame: DataFrame con datos diarios que incluye los datos horarios, la suma, la media, el máximo y el mínimo para cada día.
    """
    
    # Asegurarse de que la columna 'Time' es de tipo entero
    df['Time'] = df['Time'].astype(int)    
    df['Time_label'] = df['Time'].apply(lambda x: f"Time_{x:02d}")
    
    # Pivotar: una fila por día, columnas = horas
    df_diario = df.pivot_table(
        index=["Year", "Month", "Day"],
        columns="Time_label",
        values=variable
    ).reset_index()

    

    # Agrupar por fecha y calcular la media, el máximo y el mínimo
    # Añadir columnas estadísticas por fila
    df_diario[f"{variable}_D"] = df_diario.loc[:, df_diario.columns[3:]].sum(axis=1)   # suma de todas las horas
    df_diario["Max"]   = df_diario.loc[:, df_diario.columns[3:-1]].max(axis=1)   # máximo
    df_diario["Min"]   = df_diario.loc[:, df_diario.columns[3:-2]].min(axis=1)   # mínimo
    df_diario["Mean"]   = df_diario.loc[:, df_diario.columns[3:-3]].mean(axis=1)   # media

    df_diario['DayOfYear'] = pd.to_datetime({'year': df_diario['Year'], 'month': df_diario['Month'], 'day': df_diario['Day']}).dt.dayofyear

    return df_diario

In [6]:
df_precios_d=generate_df_variable_diaria(df_precios_h,"Price_1")

# Select relevant columns and rename
df_precios_d= df_precios_d [['Year', 'Month', 'Day', 'DayOfYear', 'Mean']].copy()
df_precios_d.rename(columns={'Mean': 'Price_mean_€_MWh'}, inplace=True)

## 3. Agregamos la POTENCIA de la celda por AÑO

In [7]:
# Sumamos la potencia renovable diaria de todas las celdas. Los valores varían or año.
df_potencia_d=df_potencia_d.groupby(['Year', 'Month', 'Day'], as_index=False)[['Pot_Solar_kW', 'Pot_Eolico_kW']].sum()

# Cambiamos unidades y renombramos columnas
df_potencia_d['Pot_Eolica_GW']=df_potencia_d['Pot_Eolico_kW']/1000000
df_potencia_d['Pot_Solar_GW']=df_potencia_d['Pot_Solar_kW']/1000000
df_potencia_d=df_potencia_d.drop(columns=['Pot_Solar_kW', 'Pot_Eolico_kW'])

## 4. Renombramos en BALANCE especificando Unidades

In [8]:
df_balance_d.rename(columns={'Demanda': 'Demanda_GWh', 'Eolica':'Gen_Eolica_GWh', 'Solar':'Gen_Solar_GWh'}, inplace=True)

## 5. Hacemos un merge de todas las tablas - inner para que se queden solo las que existan en todas

In [9]:
df_set_electrico_d = (
    df_balance_d
    .merge(df_potencia_d, on=['Year', 'Month', 'Day'], how='inner')
    .merge(df_precios_d, on=['Year', 'Month', 'Day', 'DayOfYear'], how='inner')
)
df_set_electrico_d.head()

# es posible que haya lagún registro de 2014 (el del 1 enero) los eliminamos
df_set_electrico_d.drop(df_set_electrico_d[df_set_electrico_d['Year'] == 2024].index, inplace=True)

## 6. Guardamos resultados

In [10]:
# Guardar resultados
df_set_electrico_d.to_csv(f"{BASE_DIR}/SET_ELECTRICO_D.csv", index=False)

## 7. Exploramos

In [11]:
def explora_df(df,title_1=""):
    print(f"------------EJEMPLO CABECERAS-{title_1}-------------\n--------------------------")
    print(df.head())
    print(f"------------INFO TIPO DATOS-{title_1}---------------\n--------------------------")
    df.info()
    print(f"------------ESTADÍSTICAS-{title_1} -----------------\n--------------------------")
    print(df.describe(include='all').transpose())

explora_df(df_set_electrico_d)

------------EJEMPLO CABECERAS--------------
--------------------------
   Year  Month  Day  DayOfYear  Demanda_GWh  Gen_Eolica_GWh  Gen_Solar_GWh  \
0  2019      1    1          1   547.404192       77.680013      25.586870   
1  2019      1    2          2   701.784804      194.507772      25.453782   
2  2019      1    3          3   746.345169      109.405705      22.307852   
3  2019      1    4          4   754.085459       79.098774      23.624898   
4  2019      1    5          5   681.461272      116.074127      25.952103   

   Pot_Eolica_GW  Pot_Solar_GW  Price_mean_€_MWh  
0       13.80783      3.382635         63.454583  
1       13.80783      3.382635         61.585417  
2       13.80783      3.382635         63.961667  
3       13.80783      3.382635         66.835833  
4       13.80783      3.382635         65.328750  
------------INFO TIPO DATOS----------------
--------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1826 entries, 0 to 1825
Data colu